# Investment Analysis Assignment

In [ ]:
from IPython.display import Image
Image("D:../input/image.jpg", height = '200', width = '800')

## Business and Data Understanding

Spark Funds has two minor constraints for investments:

It wants to invest between 5 to 15 million USD per round of investment.
It wants to invest only in English-speaking countries because of the ease of communication with the companies it would invest in.
For our analysis, we have to consider a country to be English speaking only if English is one of the official languages in that country.

The objective is to identify the best sectors, countries, and a suitable investment type for making investments. The overall strategy is to invest where others are investing, implying that the 'best' sectors and countries are the ones 'where most investors are investing'.

# CHECKPOINT - 1 : Data Cleaning

In [ ]:
# Import the required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

In [ ]:
# Display pandas DataFrame of floats using a format string for columns   

pd.options.display.float_format = '{:20,.2f}'.format

In [ ]:
# Use a function to change the permalink and company permalink into lower-case

def lowercase(val):
    return val.lower()

# We will use a method to remove the special character from the data sets

def remove_spe_chr(val):
    removeliteral=""
    for i in re.compile(r'[0-9a-zA-Z-+/.]').findall(val):
        removeliteral += str(i)
    return str(removeliteral)

Now load the companies and rounds data into two data frames and name them companies and rounds2 respectively.

In [ ]:
# Read the file by using the encoder

companies = pd.read_csv('D:../input/companies.txt', sep="\t", encoding = 'ISO-8859-1')
companies.head(5)

In [ ]:
rounds2 = pd.read_csv('D:../input/rounds2.csv',encoding = 'ISO-8859-1')
rounds2.head(5)

In [ ]:
# Format the fields of a dataframe into lowercase

companies["permalink"] = companies["permalink"].apply(lowercase)
rounds2["company_permalink"] = rounds2["company_permalink"].apply(lowercase)

In [ ]:
# Format the fields of a dataframe for special characters

companies["permalink"] = companies["permalink"].apply(remove_spe_chr)
rounds2["company_permalink"] = rounds2["company_permalink"].apply(remove_spe_chr)

How many unique companies are present in rounds2 ?

In [ ]:
# No. of unique companies are present in rounds2

rounds2.company_permalink.nunique()

How many unique companies are present in companies ?

In [ ]:
# No. of unique companies are present in companies

companies.permalink.nunique()

In the companies data frame, the column which can be used as the unique key for each company is

In [ ]:
# Permalink

companies['permalink'].head(5)

In [ ]:
# Any companies in the rounds2 file which are not present in companies

val1 = pd.DataFrame(rounds2.company_permalink.unique())
val2 = pd.DataFrame(companies.permalink.unique())
val2.equals(val1)

In [ ]:
# Answer yes or no: Y/N

if val2.equals(val1):
    print('Y')
else:
    print('N')

Merging Data Frames [How many observations are present in master_frame] ?

In [ ]:
# To merge the dataframes we will rename the company column name

companies.rename(columns = {'permalink':'company_permalink'}, inplace = True)

In [ ]:
# Now we will merge both the dataframes together

master_frame = pd.merge(companies, rounds2, how = 'left', on = 'company_permalink')
master_frame.head(8)

In [ ]:
# No. of observations which are present in master_fram

observations = len(master_frame)
observations

# CHECKPOINT - 2 : Funding Type Analysis

In [ ]:
# Remove the NaN values from 

master_frame = master_frame[pd.notnull(master_frame['country_code'])]
master_frame = master_frame[pd.notnull(master_frame['category_list'])]
master_frame = master_frame[pd.notnull(master_frame['raised_amount_usd'])]

The value of the investment amount for each of the four funding types are

Average funding amount of Venture , Seed , Angle & Private Equity :

In [ ]:
# For venture funding

venture_funding = master_frame[master_frame["funding_round_type"].isin(["venture"])]
print("Average funding amount of venture type : "+ str(venture_funding.raised_amount_usd.mean()))

In [ ]:
# For seed funding

seed_funding = master_frame[master_frame["funding_round_type"].isin(["seed"])]
print("Average funding amount of seed type : "+ str(seed_funding.raised_amount_usd.mean()))

In [ ]:
# For angle funding

angel_funding = master_frame[master_frame["funding_round_type"].isin(["angel"])]
print("Average funding amount of angle type : "+ str(seed_funding.raised_amount_usd.mean()))

In [ ]:
# For Private Equity

private_equity_funding = master_frame[master_frame["funding_round_type"].isin(["private_equity"])]
print("Average Private Equity Funding : "+ str(private_equity_funding.raised_amount_usd.mean()))

Considering that Spark Funds wants to invest between 5 to 15 million USD per investment round, so the investment type which is more suitable for it,

In [ ]:
# Group by the data as per the fund rounding type

funding_round_type = master_frame.groupby('funding_round_type').raised_amount_usd.mean().reset_index()
funding_round_type

In [ ]:
# Filter the data so it only contains the chosen investment type

funding_round_type = funding_round_type.loc[(funding_round_type.raised_amount_usd >= 5000000) & (funding_round_type.raised_amount_usd <= 15000000)]
funding_round_type

# CHECKPOINT - 3 : Country Analysis

In [ ]:
# The top nine countries which have received the highest total funding (across ALL sectors for the chosen investment type)

venture_cc = venture_funding.groupby("country_code").raised_amount_usd.sum().reset_index()
venture_cc.sort_values(['raised_amount_usd'], axis = 0, ascending = False, inplace = True)
top9 = venture_cc.head(9)
top9

In [ ]:
# The top three English-speaking countries in the data frame top9 are

english_country_list_codes = ['USA','AUS','CAN','IND','BMU','GBR','NZL','GIB','IRL']
top_3 = top9.loc[top9.country_code.isin(english_country_list_codes)].head(3)
top_3

# CHECKPOINT - 4 : Sector Analysis 1 

In [ ]:
# Extracting the primary sector of each category list from the category_list column

sectors_seperator = lambda x:x.split('|')[0].title()
master_frame['primary_sector'] = master_frame.category_list.apply(sectors_seperator)

In [ ]:
# Read the mapping.csv file & replace all the na & 2.na values with 0

mapping = pd.read_csv('D:../input/mapping.csv')
mapping = mapping[pd.notnull(mapping['category_list'])]
mapping.category_list = mapping.category_list.replace({'0':'na', '2.na' :'2.0'}, regex=True)
mapping.head()

In [ ]:
mapping = pd.melt(mapping, id_vars =['category_list'], value_vars =['Manufacturing','Automotive & Sports','Cleantech / Semiconductors','Entertainment','Health','News, Search and Messaging','Others','Social, Finance, Analytics, Advertising']) 
mapping.head(8)

In [ ]:
mapping = mapping[~(mapping.value == 0)]
mapping = mapping.drop('value', axis = 1)
mapping.head(8)

In [ ]:
mapping = mapping.rename(columns = {"variable":"main_sector"})
mapping.head()

In [ ]:
# Merged data frame with each primary sector mapped to its main sector

master_frame_sector = pd.merge(master_frame, mapping, how = 'inner', on = 'category_list')
master_frame_sector.head(8)

# CHECKPOINT - 5 : Sector Analysis 2

Create the three dataframes D1, D2 and D3

In [ ]:
# For D1

D1_SA2 = master_frame_sector.loc[(master_frame_sector.country_code == "USA") & (master_frame_sector.funding_round_type == "venture") & 
                             (master_frame_sector.raised_amount_usd >= 5000000) & (master_frame_sector.raised_amount_usd <= 15000000), :]

In [ ]:
# The total amount invested for each main sector in a separate column

D1_total_sum = D1_SA2.groupby("main_sector").raised_amount_usd.sum().sort_values(ascending = False).to_frame(name='total_sum_invested')

In [ ]:
# The total number (or count) of investments for each main sector in a separate column

D1_total_count = D1_SA2.groupby("main_sector").raised_amount_usd.count().sort_values(ascending = False).to_frame(name='count')

In [ ]:
# Merging frames to created final D1

D1_SA2 = pd.merge(D1_SA2, D1_total_count, how='inner', on='main_sector')
D1 = pd.merge(D1_SA2, D1_total_sum, how='inner', on='main_sector')
D1.head(8)

In [ ]:
# For D2

D2_SA2 = master_frame_sector.loc[(master_frame_sector.country_code=="GBR") & (master_frame_sector.funding_round_type=="venture") & 
                             (master_frame_sector.raised_amount_usd >=5000000) & (master_frame_sector.raised_amount_usd <= 15000000), :]

In [ ]:
# The total amount invested for each main sector in a separate column

D2_total_sum = D2_SA2.groupby("main_sector").raised_amount_usd.sum().sort_values(ascending = False).to_frame(name='total_sum_invested')

In [ ]:
# The total number (or count) of investments for each main sector in a separate column

D2_total_count = D2_SA2.groupby("main_sector").raised_amount_usd.count().sort_values(ascending = False).to_frame(name='count')

In [ ]:
# Merging frames to created final D2

D2_SA2 = pd.merge(D2_SA2, D2_total_count, how='inner', on='main_sector')
D2 = pd.merge(D2_SA2, D2_total_sum, how='inner', on='main_sector')
D2.head(8)

In [ ]:
# For D3

D3_SA2 = master_frame_sector.loc[(master_frame_sector.country_code == "IND") & (master_frame_sector.funding_round_type == "venture") & 
                                  (master_frame_sector.raised_amount_usd >= 5000000) & (master_frame_sector.raised_amount_usd <= 15000000), :]

In [ ]:
# The total amount invested for each main sector in a separate column

D3_total_sum = D3_SA2.groupby("main_sector").raised_amount_usd.sum().sort_values(ascending = False).to_frame(name='total_sum_invested')

In [ ]:
# The total number (or count) of investments for each main sector in a separate column

D3_total_count = D3_SA2.groupby("main_sector").raised_amount_usd.count().sort_values(ascending = False).to_frame(name='count')

In [ ]:
# Merging frames to created final D3

D3_SA2 = pd.merge(D3_SA2, D3_total_count, how='inner', on='main_sector')
D3 = pd.merge(D3_SA2, D3_total_sum, how='inner', on='main_sector')
D3.head(8)

Sector-wise Investment Analysis

In [ ]:
# 1. Total number of investments (count)

print("Total number of investments for Country1 : " + str(D1.raised_amount_usd.count()))
print("Total number of investments for Country2 : " + str(D2.raised_amount_usd.count()))
print("Total number of investments for Country3 : " + str(D3.raised_amount_usd.count()))

In [ ]:
# 2. Total amount of investment (USD)

print("Total amount of investments for Country1 : " + str(D1.raised_amount_usd.sum()))
print("Total amount of investments for Country2 : " + str(D2.raised_amount_usd.sum()))
print("Total amount of investments for Country3 : " + str(D3.raised_amount_usd.sum()))

In [ ]:
#  3.Top sector (based on count of investments) for all the 3 countries

D1_total_count_investments = D1.groupby("main_sector").total_sum_invested.count().sort_values(ascending = False).to_frame(name = 'total_count')
print("Top 3 sector for Country1 :")
print(D1_total_count_investments.head(3))
Country1 = D1_total_count_investments.sum()
print(Country1)

In [ ]:
D2_total_count_investments = D2.groupby("main_sector").total_sum_invested.count().sort_values(ascending = False).to_frame(name = 'total_count')
print("Top 3 sector for Country2 :")
print(D2_total_count_investments.head(3))
Country2 = D2_total_count_investments.sum()
print(Country2)

In [ ]:
D3_total_count_investments = D3.groupby("main_sector").total_sum_invested.count().sort_values(ascending = False).to_frame(name='total_count')
print("Top 3 sector for Country3 :")
print(D3_total_count_investments.head(3))
Country3 = D3_total_count_investments.sum()
print(Country3)

In [ ]:
print("Top 3 sector for Country1 :")
D1_total_count_investments.head(3)

In [ ]:
print("Top 3 sector for Country2 :")
D2_total_count_investments.head(3)

In [ ]:
print("Top 3 sector for Country3 :")
D3_total_count_investments.head(3)

In [ ]:
#  4. Second-best sector (based on count of investments)

print("Second best sector for Country1 :")
D1_total_count_investments.iloc[1:2]

In [ ]:
print("Second best sector for Country2 :")
D2_total_count_investments.iloc[1:2]

In [ ]:
print("Second best sector for Country3 :")
D3_total_count_investments.iloc[1:2]

In [ ]:
# 5. Third-best sector (based on count of investments)

print("Third best sector for Country1 :")
D1_total_count_investments.iloc[2:3]

In [ ]:
print("Third best sector for Country2 :")
D2_total_count_investments.iloc[2:3]

In [ ]:
print("Third best sector for Country3 :")
D3_total_count_investments.iloc[2:3]

In [ ]:
# 6. Number of investments in the top sector (refer to point 3)

Country1 = D1_total_count_investments.iloc[:1]
Country1

In [ ]:
Country2 = D2_total_count_investments.iloc[:1]
Country2

In [ ]:
Country3 = D3_total_count_investments.iloc[:1]
Country3

In [ ]:
# 7. Number of investments in the second-best sector (refer to point 4)

Country1 = D1_total_count_investments.iloc[1:2]
Country1

In [ ]:
Country2 = D2_total_count_investments.iloc[1:2]
Country2

In [ ]:
Country3 = D2_total_count_investments.iloc[1:2]
Country3

In [ ]:
# 8. Number of investments in the third-best sector (refer to point 5)

Country1 = D3_total_count_investments.iloc[2:3]
Country1

In [ ]:
Country2 = D2_total_count_investments.iloc[2:3]
Country2

In [ ]:
Country3 = D3_total_count_investments.iloc[2:3]
Country3

In [ ]:
# 9. For the top sector count-wise (point 3), which company received the highest investment?

D1_val = D1[D1.main_sector == "Others"]
D1_val = D1_val.groupby(["company_permalink","name"]).raised_amount_usd.sum().reset_index()
D1_val.sort_values(["raised_amount_usd"], axis = 0, ascending = False, inplace = True)
d1_top_list = (D1_val.iloc[0:1, 1:3].values.tolist())[0]

D2_val = D2[D2.main_sector == "Others"]
D2_val = D2_val.groupby(["company_permalink","name"]).raised_amount_usd.sum().reset_index()
D2_val.sort_values(["raised_amount_usd"], axis = 0, ascending = False, inplace = True)
d2_top_list = (D2_val.iloc[0:1, 1:3].values.tolist())[0]

D3_val = D3[D3.main_sector == "Others"]
D3_val = D3_val.groupby(["company_permalink","name"]).raised_amount_usd.sum().reset_index()
D3_val.sort_values(["raised_amount_usd"], axis = 0, ascending = False, inplace = True)
d3_top_list = (D3_val.iloc[0:1, 1:3].values.tolist())[0]

print(" '{0}' company recived highest investments for Country1 worth total (sum) $".format(d1_top_list[0]),d1_top_list[1])
print(" '{0}' company recived highest investments for Country2 worth total (sum) $".format(d2_top_list[0]),d2_top_list[1])
print(" '{0}' company recived highest investments for Country3 worth total (sum) $".format(d3_top_list[0]),d3_top_list[1])

In [ ]:
# 10. For the second-best sector count-wise (point 4), which company received the highest investment?

D1_val = D1[D1.main_sector == "Social, Finance, Analytics, Advertising"]
D1_val = D1_val.groupby(["company_permalink","name"]).raised_amount_usd.sum().reset_index()
D1_val.sort_values(["raised_amount_usd"], axis = 0, ascending = False, inplace = True)
d1_top_list = (D1_val.iloc[0:1, 1:3].values.tolist())[0]

D2_val = D2[D2.main_sector == "Social, Finance, Analytics, Advertising"]
D2_val = D2_val.groupby(["company_permalink","name"]).raised_amount_usd.sum().reset_index()
D2_val.sort_values(["raised_amount_usd"], axis = 0, ascending = False, inplace = True)
d2_top_list = (D2_val.iloc[0:1, 1:3].values.tolist())[0]

D3_val = D3[D3.main_sector == "Social, Finance, Analytics, Advertising"]
D3_val = D3_val.groupby(["company_permalink","name"]).raised_amount_usd.sum().reset_index()
D3_val.sort_values(["raised_amount_usd"], axis = 0, ascending = False, inplace = True)
d3_top_list = (D3_val.iloc[0:1, 1:3].values.tolist())[0]

print(" '{0}' company recived highest investments for Country1 worth total (sum) $".format(d1_top_list[0]),d1_top_list[1])
print(" '{0}' company recived highest investments for Country2 worth total (sum) $".format(d2_top_list[0]),d2_top_list[1])
print(" '{0}' company recived highest investments for Country3 worth total (sum) $".format(d3_top_list[0]),d3_top_list[1])

# CHECKPOINT - 6 : Plots

In [ ]:
# A plot showing the fraction of total investments (globally) in angel, venture, seed, and private equity, and the average amount of investment in each funding type.

plotting_frame = master_frame_sector[master_frame_sector["funding_round_type"].isin(["angle","venture","seed","private_equity"])]
plotting_frame = plotting_frame.loc[(plotting_frame.raised_amount_usd >=5000000) & (plotting_frame.raised_amount_usd <= 15000000),:]
sns.violinplot(x = 'funding_round_type', y = 'raised_amount_usd', split = True, inner = "quart", data = plotting_frame)
sns.despine(left = True)
plt.title('Fraction of total investments')
plt.yscale('log')
plt.show()

In [ ]:
# A plot showing the top 9 countries against the total amount of investments of funding type FT.

plotting = top9.set_index("country_code")
plotting.plot.bar(logy = True);
plotting

In [ ]:
# A plot showing the number of investments in the top 3 sectors of the top 3 countries on one chart (for the chosen investment type FT).

D1_plot = D1
D1_plot = D1_plot.groupby("main_sector").raised_amount_usd.count().reset_index()
D1_plot.sort_values(["raised_amount_usd"], axis = 0, ascending = False, inplace = True)
D1_plot = D1_plot.head(3)

D2_plot = D2
D2_plot = D2_plot.groupby("main_sector").raised_amount_usd.count().reset_index()
D2_plot.sort_values(["raised_amount_usd"], axis = 0, ascending = False, inplace = True)
D2_plot = D2_plot.head(3)

D3_plot = D3
D3_plot = D3_plot.groupby("main_sector").raised_amount_usd.count().reset_index()
D3_plot.sort_values(["raised_amount_usd"], axis = 0, ascending = False, inplace = True)
D3_plot = D3_plot.head(3)

In [ ]:
D12 = pd.merge(D1_plot, D2_plot, how = 'outer', on = 'main_sector')
D123 = pd.merge(D12, D3_plot, how = 'outer', on = 'main_sector')

In [ ]:
D123 = D123.rename(columns = {"raised_amount_usd_x": "USD", "raised_amount_usd_y": "GBR" ,"raised_amount_usd": "IND"})
D123= D123.set_index("main_sector")
D123.fillna(0)

In [ ]:
D123.T.plot.bar(logy = True)

# END